# EXAMPLE 3
## Explaining the experiment
In this example, we are investigating a 128kx8 SRAM that was exposed to radiation in static tests with different bias voltages and with the same pattern, 0x55. Several irradiations were done yielding the CSV files present in this folder. 

According to the manufacturer, MBUs are impossible due to the presence of interleaving. Besides, the technology was so old that it was very difficult to induce MCUs in the memory with the conditions of the experiment.

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
### Optional
push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl"); # <-- ADAPT THIS INSTRUCTION TO YOUR COMPUTER!
# If you are a Windows user, remember that subfolders are indicated with \\ or /, NEVER with a simple backslash.

In [ ]:
using DelimitedFiles, LELAPE

## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 8 bits
* Memory size in words: 128k is just 2^17.
* In SRAMs, it seems more likely to succeed the XOR operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [ ]:
LA = 2^17 # Memory size in words
WordWidth = 8 # Selfexplaining.
Operation = "XOR" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [ ]:
DATA10 = readdlm("DATA/ExampleSRAM14.csv", ',', UInt32, '\n', skipstart=1)

Good!! If you have correctly proceeded, a 902x4 unsigned integer matrix is loaded. Last column is useless and could have been omitted. At any rate, the software will solve the problem. Now, let us analyze the DATA10 set. First of all, the MBUs.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ function that returns the MBUs present in DATA. Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [ ]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA10[:,2], DATA10[:,3], WordWidth)

The following loop will show how many MBUs per number of flipped bits were observed:

In [ ]:
for size = 1: WordWidth
    println("$size-bit MBUs: ", length(findall(MBUSize.==size)))
end

__WHAT???? ARE THERE 2-BIT MCUS?__ This is nonsense since this is physically impossible. Perhaps the problem comes from the fact that the memory was overirradiated. Let us calculate the expected number of false 2-bit MBUs...

In [ ]:
NF2BITMBU = NF2BitMCUs(DATA10, LA, "MBU", WordWidth, WordWidth)
println("We expected $NF2BITMBU false 2-bit MBUs in this  experiment.")

Therefore, we expected in average 2.71... false 2-bit MBUs and observed 3. This makes sense.

_NOTE: This value was got from ExampleSRAM10. If you change the data file, this value also does_

## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [ ]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C10_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C10_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C10_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C10_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [ ]:
C10_SCY, C10_MCU, C10_TRC, C10_SHF = DetectAnomalies_FullCheck(DATA10, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)

########

println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C10_SCY[:, 1])
    println("Value: 0x", string(C10_SCY[index, 1], base=16, pad = 6), " --> ", Int(C10_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA10, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

### What has happened? 
All the returned matrices are void. The reason is that there are not MCUs, only SBUs, or at least are so scarce that cannot be distinguished from statistical deviations. Now, we can conclude the analysis since all the bitflips are SBUs.

## Further analysis
You can find in this folder other actual data got from the same memory. Analyze them and, if you wish. For your information, there are at least some files that yield some positive results.

Only ExampleSRAM14.csv managed to show the following anomaly:

* 0x000010

if you wish, you can load other data value and check if there were MCUs using this signature.

In [ ]:
Anomalies = [0x000010] # The only detected anomaly in vector format.
Labeled_addresses = MCU_Indexes(DATA10, Operation, Anomalies, UsePseudoAddress, WordWidth)
Events = Classify_Addresses_in_MCU(DATA10, Labeled_addresses, UsePseudoAddress, WordWidth)
####
# This helps to read data.
NBitFlips = 0; # Let us profit this loop to get the number of bitflips.
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    NBitFlips += NMCUs*(length(Events)-k+1)
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    for row = 1:NMCUs
        for bit = 1:length(Events)-k+1
            print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
            bit != length(Events)-k+1 ? print(", ") : print("\n")

        end
    end
    println()
end

As there are a lot of events, it is worth checking if they are just false 2-bit MCUs.

In [ ]:
NF_2b_MCUs = NF2BitMCUs(NBitFlips, LA, Operation, length(Anomalies), WordWidth, UsePseudoAddress)
println("The number of expected false 2-bit MCUs is $NF_2b_MCUs. Use this to validate the previous result.")

How to validate this result? We usually recommend to adapt Eq. 8 in J. L. Autran, D. Munteanu, P. Roche, and G. Gasiot, _“Real-time softerror rate measurements: A review,”_ Microelectronics Reliability, vol. 54, no. 8, pp. 1455 – 1476, 2014 to your experiments. Inverse χ² function is ___XXX___

In [ ]:
# A statistical package is necessary. If not installed, uncomment the following two lines.
##using Pkg
##Pkg.add("StatsFuns")
using StatsFuns

In [ ]:
confidence = 0.95 # 95% of confidence
NMCU2 = length(Events[end-1][:,1]) # Let us see how many 2-bit MCUs were detected.
NMCU2 == 0 ? NMCU2 = eps() : NMCU2 = NMCU2 # This avoids using 0 in the inverse χ² function.

upperbound = 0.5*chisqinvcdf( 2*NMCU2+2, 0.5+0.5*confidence)
lowerbound = 0.5*chisqinvcdf( 2*NMCU2, 0.5-0.5*confidence)

println("You got $NMCU2 2-bit MCUs, so the expected mean number is between $lowerbound and $upperbound events.")
println("The number of expected false 2-bit MCUs is $NF_2b_MCUs.")
if !(lowerbound < NF_2b_MCUs < upperbound)
    println("Some of the 2-bit MCUs are REAL.")
else
    println("Think again if your 2-bit MCUs really occurred.")
end

# An interesting detail
These results were used for G. Korkian et al., _"Experimental and Analytical Study of the Responses of Nanoscale Devices to Neutrons Impinging at Various Incident Angles,"_ IEEE Transactions on Nuclear Science, vol. 67, no. 11, pp. 2345-2352, Nov. 2020, doi: 10.1109/TNS.2020.3025104. 